In [1]:
from helper import get_openai_api_key, get_llama_cloud_api_key
from IPython.display import display, HTML
from helper import extract_html_content
from llama_index.utils.workflow import draw_all_possible_flows
import os

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
llama_cloud_api_key = get_llama_cloud_api_key()
openai_api_key = get_openai_api_key()

In [4]:
from llama_parse import LlamaParse

In [5]:
documents = LlamaParse(
    api_key=llama_cloud_api_key,
    base_url=os.getenv("LLAMA_CLOUD_BASE_URL"),
    result_type="markdown",
    content_guideline_instruction="This is a resume, gather related facts together and format it as bullet points with headers"
).load_data(
    "data/fake_resume.pdf",
)

Started parsing the file under job_id 4ec7c0bb-a320-4fe5-98de-1d6c2942d332


In [6]:
print(documents[2].text)

# Projects

# EcoTrack | GitHub

- Built full-stack application for tracking carbon footprint using React, Node.js, and MongoDB
- Implemented machine learning algorithm for providing personalized sustainability recommendations
- Featured in TechCrunch's "Top 10 Environmental Impact Apps of 2023"

# ChatFlow | Demo

- Developed real-time chat application using WebSocket protocol and React
- Implemented end-to-end encryption and message persistence
- Serves 5000+ monthly active users

# Certifications

- AWS Certified Solutions Architect (2023)
- Google Cloud Professional Developer (2022)
- MongoDB Certified Developer (2021)

# Languages

- English (Native)
- Mandarin Chinese (Fluent)
- Spanish (Intermediate)

# Interests

- Open source contribution
- Tech blogging (15K+ Medium followers)
- Hackathon mentoring
- Rock climbing


In [7]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex

In [8]:
index = VectorStoreIndex.from_documents(
    documents,
    embed_model=OpenAIEmbedding(model_name="text-embedding-3-small", 
                                api_key= openai_api_key)
)

In [9]:
from llama_index.llms.openai import OpenAI

In [10]:
llm = OpenAI(model="gpt-4o-mini")

In [11]:
query_engine = index.as_query_engine(llm=llm, similarity_top_k=5)
response = query_engine.query("What is this person's name and what was their most recent job?")
print(response)

The person's name is Sarah Chen, and their most recent job is Senior Full Stack Developer at TechFlow Solutions.


In [12]:
storage_dir = "./storage"

index.storage_context.persist(persist_dir=storage_dir)

In [13]:
from llama_index.core import StorageContext, load_index_from_storage

In [14]:
# Check if the index is stored on disk
if os.path.exists(storage_dir):
    # Load the index from disk
    storage_context = StorageContext.from_defaults(persist_dir=storage_dir)
    restored_index = load_index_from_storage(storage_context)
else:
    print("Index not found on disk.")

In [15]:
response = restored_index.as_query_engine().query("What is this person's name and what was their most recent job?")
print(response)

This person's name is Sarah Chen and their most recent job was as a Senior Full Stack Developer at TechFlow Solutions in San Francisco, CA.


In [16]:
from llama_index.core.tools import FunctionTool
from llama_index.core.agent import FunctionCallingAgent

In [ ]:
def query_resume(q: str) -> str:
    """Answers questions about a specific resume."""
    response = query_engine.query(f"This is a question about the specific resume we have in our database: {q}")
    return response.response

In [18]:
resume_tool = FunctionTool.from_defaults(fn=query_resume)

In [19]:
agent = FunctionCallingAgent.from_tools(
    tools=[resume_tool],
    llm=llm,
    verbose=True
)

In [20]:
response = agent.chat("How many years of experience does the applicant have?")
print(response)

> Running step 7dcee0e6-7623-4901-a93d-4e9bc6d1358d. Step input: How many years of experience does the applicant have?
Added user message to memory: How many years of experience does the applicant have?
=== Calling Function ===
Calling function: query_resume with args: {"q": "How many years of experience does the applicant have?"}
=== Function Output ===
The applicant has over 6 years of experience as a Full Stack Web Developer.
> Running step 8d351f42-76fe-4952-ac5a-35f22526c293. Step input: None
=== LLM Response ===
The applicant has over 6 years of experience as a Full Stack Web Developer.
The applicant has over 6 years of experience as a Full Stack Web Developer.


In [21]:
from llama_index.core.workflow import (
    StartEvent,
    StopEvent,
    Workflow,
    step,
    Event,
    Context
)

In [22]:
class QueryEvent(Event):
    query: str

In [ ]:
class RAGWorkflow(Workflow):
    storage_dir = "./storage"
    llm: OpenAI
    query_engine: VectorStoreIndex

    @step
    async def set_up(self, ctx: Context, ev: StartEvent) -> QueryEvent:

        if not ev.resume_file:
            raise ValueError("No resume file provided")

        self.llm = OpenAI(model="gpt-4o-mini")

        if os.path.exists(self.storage_dir):
            storage_context = StorageContext.from_defaults(persist_dir=self.storage_dir)
            index = load_index_from_storage(storage_context)
        else:
            documents = LlamaParse(
                result_type="markdown",
                content_guideline_instruction="This is a resume, gather related facts together and format it as bullet points with headers"
            ).load_data(ev.resume_file)

            index = VectorStoreIndex.from_documents(
                documents,
                embed_model=OpenAIEmbedding(model_name="text-embedding-3-small")
            )
            index.storage_context.persist(persist_dir=self.storage_dir)

        self.query_engine = index.as_query_engine(llm=self.llm, similarity_top_k=5)
        return QueryEvent(query=ev.query)

    @step
    async def ask_question(self, ctx: Context, ev: QueryEvent) -> StopEvent:
        response = self.query_engine.query(f"This is a question about the specific resume we have in our database: {ev.query}")
        return StopEvent(result=response.response)

In [24]:
w = RAGWorkflow(timeout=120, verbose=False)
result = await w.run(
    resume_file="./data/fake_resume.pdf",
    query="Where is the first place the applicant worked?"
)
print(result)

The first place the applicant worked is StartupHub in San Jose, CA.


In [25]:
WORKFLOW_FILE = "workflows/rag_workflow.html"
draw_all_possible_flows(w, filename=WORKFLOW_FILE)
html_content = extract_html_content(WORKFLOW_FILE)
display(HTML(html_content), metadata=dict(isolated=True))

<class 'NoneType'>
<class 'llama_index.core.workflow.events.StopEvent'>
<class '__main__.QueryEvent'>
workflows/rag_workflow.html
